**Scraping our first page with Requests**

In [1]:
import requests

In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

**Parsing HTML links with BeautifulSoup**

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select("table.stats_table")[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if "/squads/" in l]

In [6]:
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

['https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/s

**Extract match stats using Pandas and Requests**

In [7]:
data = requests.get(team_urls[0])

In [8]:
import pandas as pd
from io import StringIO

matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (4),1 (1),Manchester City,NaN,NaN,45.0,81145.0,Martin Ødegaard,4-3-3,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,0.8,1.2,78.0,59984.0,Martin Ødegaard,4-3-3,Michael Oliver,Match Report,NaN
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,2.0,1.0,53.0,24189.0,Martin Ødegaard,4-3-3,David Coote,Match Report,NaN
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,3.2,0.6,71.0,59961.0,Martin Ødegaard,4-3-3,Paul Tierney,Match Report,NaN
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,2.3,0.9,55.0,60192.0,Martin Ødegaard,4-3-3,Anthony Taylor,Match Report,NaN


**Get match shooting stats with Requests and Pandas**

In [9]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and "all_comps/shooting/" in l]
links

['/en/squads/18bb7c10/2023-2024/matchlogs/all_comps/shooting/Arsenal-Match-Logs-All-Competitions',
 '/en/squads/18bb7c10/2023-2024/matchlogs/all_comps/shooting/Arsenal-Match-Logs-All-Competitions',
 '/en/squads/18bb7c10/2023-2024/matchlogs/all_comps/shooting/Arsenal-Match-Logs-All-Competitions',
 '/en/squads/18bb7c10/2023-2024/matchlogs/all_comps/shooting/Arsenal-Match-Logs-All-Competitions']

In [10]:
data = requests.get(f"https://fbref.com{links[0]}")

In [11]:
shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]

**Cleaning and merging scraped data with Pandas**

In [12]:
shooting.head()

For Arsenal                                                              \
         Date   Time              Comp                Round  Day    Venue   
0  2023-08-06  16:00  Community Shield  FA Community Shield  Sun  Neutral   
1  2023-08-12  12:30    Premier League          Matchweek 1  Sat     Home   
2  2023-08-21  20:00    Premier League          Matchweek 2  Mon     Away   
3  2023-08-26  15:00    Premier League          Matchweek 3  Sat     Home   
4  2023-09-03  16:30    Premier League          Matchweek 4  Sun     Home   

                                         ... Standard               Expected  \
  Result     GF     GA         Opponent  ...     Dist   FK PK PKatt       xG   
0      D  1 (4)  1 (1)  Manchester City  ...      NaN  NaN  0     0      NaN   
1      W      2      1  Nott'ham Forest  ...     19.1  0.0  0     0      0.8   
2      W      1      0   Crystal Palace  ...     16.4  0.0  1     1      2.0   
3      D      2      2           Fulham  ...     13.8  0.0  1     1      3.2   
4      W      3      1   Manchester Utd  ...     15.0  0.0  0     0      2.3   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN     NaN  NaN     NaN        Match Report  
1  0.8    0.06  1.2     1.2        Match Report  
2  1.2    0.09 -1.0    -1.2        Match Report  
3  2.4    0.14 -1.2    -1.4        Match Report  
4  2.3    0.13  0.7     0.7        Match Report  

[5 rows x 26 columns]

In [13]:
shooting.columns = shooting.columns.droplevel()
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (4),1 (1),Manchester City,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,...,19.1,0.0,0,0,0.8,0.8,0.06,1.2,1.2,Match Report
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,16.4,0.0,1,1,2.0,1.2,0.09,-1.0,-1.2,Match Report
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,13.8,0.0,1,1,3.2,2.4,0.14,-1.2,-1.4,Match Report
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,...,15.0,0.0,0,0,2.3,2.3,0.13,0.7,0.7,Match Report


In [14]:
shooting["Date"]

0     2023-08-06
1     2023-08-12
2     2023-08-21
3     2023-08-26
4     2023-09-03
5     2023-09-17
6     2023-09-20
7     2023-09-24
8     2023-09-27
9     2023-09-30
10    2023-10-03
11    2023-10-08
12    2023-10-21
13    2023-10-24
14    2023-10-28
15    2023-11-01
16    2023-11-04
17    2023-11-08
18    2023-11-11
19    2023-11-25
20    2023-11-29
21    2023-12-02
22    2023-12-05
23    2023-12-09
24    2023-12-12
25    2023-12-17
26    2023-12-23
27    2023-12-28
28    2023-12-31
29    2024-01-07
30    2024-01-20
31    2024-01-30
32    2024-02-04
33    2024-02-11
34    2024-02-17
35    2024-02-21
36    2024-02-24
37    2024-03-04
38    2024-03-09
39    2024-03-12
40           NaN
Name: Date, dtype: object

In [15]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (4),1 (1),Manchester City,...,4-3-3,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...,7,3,NaN,NaN,0,0
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,...,4-3-3,Michael Oliver,Match Report,NaN,15,7,19.1,0.0,0,0
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,4-3-3,David Coote,Match Report,NaN,13,2,16.4,0.0,1,1
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,4-3-3,Paul Tierney,Match Report,NaN,18,9,13.8,0.0,1,1
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,...,4-3-3,Anthony Taylor,Match Report,NaN,17,5,15.0,0.0,0,0


**Scraping data for multiple seasons and teams with a loop**

In [16]:
years = list(range(2024, 2022, -1))
all_matches = []

In [17]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [18]:
import time

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select("table.stats_table")[0]
    
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if "/squads/" in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com/{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split('/')[-1].replace("-Stats",'').replace('-',' ')

        data = requests.get(team_url)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = soup.find_all('a')
        links = [l.get("href") for l in links]
        links = [l for l in links if l and "all_comps/shooting/" in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        
        all_matches.append(team_data)
        time.sleep(5)


In [19]:
len(all_matches)

40

In [20]:
match_df = pd.concat(all_matches)

In [21]:
match_df.columns = [c.lower() for c in match_df.columns]
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2024,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2024,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2024,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2024,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1,0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2024,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,2023-04-30,14:00,Premier League,Matchweek 34,Sun,Away,L,1,3,Newcastle Utd,...,Match Report,NaN,4.0,3.0,17.3,0.0,0,0,2023,Southampton
43,2023-05-08,20:00,Premier League,Matchweek 35,Mon,Away,L,3,4,Nott'ham Forest,...,Match Report,NaN,18.0,4.0,14.0,0.0,1,1,2023,Southampton
44,2023-05-13,15:00,Premier League,Matchweek 36,Sat,Home,L,0,2,Fulham,...,Match Report,NaN,5.0,1.0,24.2,0.0,0,0,2023,Southampton
45,2023-05-21,14:00,Premier League,Matchweek 37,Sun,Away,L,1,3,Brighton,...,Match Report,NaN,5.0,1.0,13.8,1.0,0,0,2023,Southampton


In [22]:
match_df.to_csv("matches.csv")